In [ ]:
# Copyright 2019 Google Inc. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

# Setup Summary

* Pre-condition
    - You have a [GCP project setup](https://www.kubeflow.org/docs/gke/deploy/project-setup/) for your Kubeflow Deployment with you having the [owner role](https://cloud.google.com/iam/docs/understanding-roles#primitive_role_definitions) for the project and with the following APIs enabled:
        - [Compute Engine API](https://pantheon.corp.google.com/apis/library/compute.googleapis.com)
        - [Kubernetes Engine API](https://pantheon.corp.google.com/apis/library/container.googleapis.com)
        - [Identity and Access Management(IAM) API](https://pantheon.corp.google.com/apis/library/iam.googleapis.com)
        - [Deployment Manager API](https://pantheon.corp.google.com/apis/library/deploymentmanager.googleapis.com)
        - [Cloud Resource Manager API](https://pantheon.corp.google.com/apis/library/cloudresourcemanager.googleapis.com)
        - [Cloud Filestore API](https://pantheon.corp.google.com/apis/library/file.googleapis.com)
        - [AI Platform Training & Prediction API](https://pantheon.corp.google.com/apis/library/ml.googleapis.com)
    - You have set up [OAuth for Cloud IAP](https://www.kubeflow.org/docs/gke/deploy/oauth-setup/)
    - You have installed and setup [kubectl](https://kubernetes.io/docs/tasks/tools/install-kubectl/)
    - You have installed [gcloud-sdk](https://cloud.google.com/sdk/)

* Deploy a kubeflow cluster through [CLI](https://www.kubeflow.org/docs/gke/deploy/deploy-cli/)
    - `NOTE` : The [UI](https://www.kubeflow.org/docs/gke/deploy/deploy-ui/) method of deploying a kubeflow cluster does not support Kubeflow v0.7 yet
    - Download and install kfctl
    - Create user credentials
    - Setup environment variables

* Create service account
```bash
export SA_NAME = [service account name]
gcloud iam service-accounts create ${SA_NAME}
gcloud projects add-iam-policy-binding ${PROJECT_ID} \
    --member serviceAccount:${SA_NAME}@${PROJECT_ID}.iam.gserviceaccount.com \
    --role 'roles/editor'
gcloud iam service-accounts keys create ~/key.json \
    --iam-account ${SA_NAME}@${PROJECT_ID}.iam.gserviceaccount.com
```

* Authorize for Source Repository
```bash
gcloud auth configure-docker
```

* Update local kubeconfig (for submiting job to kubeflow cluster)
```bash
export CLUSTER_NAME=${DEPLOYMENT_NAME} # this is the deployment name or the kubenete cluster name
export ZONE=us-central1-c
gcloud container clusters get-credentials ${CLUSTER_NAME} --region ${ZONE}
```

* Set the environmental variable: GOOGLE_APPLICATION_CREDENTIALS
```bash
export GOOGLE_APPLICATION_CREDENTIALS = ....
```
```python
os.environ['GOOGLE_APPLICATION_CREDENTIALS']=...
```

* Install the lastest version of kfp
```python
pip install kfp
```

* Create an OAuth client ID credentials of type Other according to the tutorial [here](
https://cloud.google.com/iap/docs/authentication-howto#authenticating_from_a_desktop_app)

**Note: the setup is only required for running notebook outside kubeflow cluster. Most of the setup should alreayd be done for kubeflow pipeline notebook.**

## Set up Kubeflow Cluster

### Deploy Kubeflow Cluster

In [ ]:
## Download kfctl v0.7.0
! curl -LO https://github.com/kubeflow/kubeflow/releases/download/v0.7.0/kfctl_v0.7.0_linux.tar.gz
    
## Unpack the tar ball
! tar -xvf kfctl_v0.7.0_linux.tar.gz

In [ ]:
## Create user credentials
! gcloud auth application-default login

### Set up environment variables

In [1]:
## Setup environment variables - Make sure to replace the <...> with actual values

# Set your GCP project ID and the zone where you want to create the Kubeflow deployment
%env PROJECT=<your gcp project id>
%env ZONE=<gcp zone to launch kubeflow cluster>

# google cloud storage bucket
%env GCP_BUCKET=<cloud storage bucket>

# Use the following kfctl configuration file for authentication with 
# Cloud IAP (recommended):
uri = "https://raw.githubusercontent.com/kubeflow/manifests/v0.7-branch/kfdef/kfctl_gcp_iap.0.7.0.yaml"
uri = uri.strip()
%env CONFIG_URI=$uri

# For using Cloud IAP for authentication, create environment variables
# from the OAuth client ID and secret that you obtained earlier:
%env CLIENT_ID=<your OAuth client id>
%env CLIENT_SECRET=<your OAuth client secret>

# Set KF_NAME to the name of your Kubeflow deployment. You also use this
# value as directory name when creating your configuration directory. 
# See the detailed description in the text below this code snippet.
# For example, your deployment name can be 'my-kubeflow' or 'kf-test'.
%env KF_NAME=<kubeflow deployment name>

# Set up name of the service account that should be created and used
# while creating the Kubeflow cluster
%env SA_NAME=<name of SA to be created for deploying Kubeflow>

SyntaxError: invalid syntax (<ipython-input-1-0b4d092f4c6e>, line 4)

In [ ]:
## Use environment variables set up above to update configs
! gcloud config set project ${PROJECT}

! gcloud config set compute/zone ${ZONE}


# Set the path to the base directory where you want to store one or more 
# Kubeflow deployments. For example, /opt/.
# Here we use the current working directory as the base directory
# Then set the Kubeflow application directory for this deployment.

import os
base = os.getcwd()
%env BASE_DIR=$base

kf_dir = os.getenv('BASE_DIR') + "/" + os.getenv('KF_NAME')
%env KF_DIR=$kf_dir

# The following command is optional. It adds the kfctl binary to your path.
# If you don't add kfctl to your path, you must use the full path
# each time you run kfctl. In this example, the kfctl file is present in
# the current directory
new_path = os.getenv('PATH') + ":" + os.getenv('BASE_DIR')
%env PATH=$new_path

### Create service account


In [ ]:
! gcloud iam service-accounts create ${SA_NAME}
! gcloud projects add-iam-policy-binding ${PROJECT} \
  --member serviceAccount:${SA_NAME}@${PROJECT}.iam.gserviceaccount.com \
  --role 'roles/editor'
! gcloud iam service-accounts keys create key.json \
  --iam-account ${SA_NAME}@${PROJECT}.iam.gserviceaccount.com

### Set GOOGLE_APPLICATION_CREDENTIALS

In [ ]:
key_path = os.getenv('BASE_DIR') + "/" + 'key.json'
%env GOOGLE_APPLICATION_CREDENTIALS=$key_path

### Install kfp

In [ ]:
! pip3 install kfp --upgrade --user

### Setup and deploy Kubeflow

In [ ]:
! mkdir -p ${KF_DIR}
%cd $kf_dir
! kfctl apply -V -f ${CONFIG_URI}

# Sanity Check

In [ ]:
! kubectl -n istio-system describe ingress